In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cPickle, h5py, os

from oc_ica import analysis
reload(analysis)

Using gpu device 0: Tesla K40c (CNMeM is enabled with initial size: 20.0% of memory, cuDNN 4007)


<module 'oc_ica.analysis' from '/home/jesse/Development/oc_ica/oc_ica/analysis.pyc'>

# HDF5 Storage

In [2]:
folder = '/home/jesse/Development/results/oc_ica'
a_file = 'a_array-128_OC-2.0_priors-L2_L4_RANDOM_COHERENCE.h5'
fit_files = ['comparison_mixtures-128_OC-2.0_priors-L2_L4_RANDOM_COHERENCE_models-2.h5',
             'comparison_mixtures-128_OC-2.0_priors-L2_L4_RANDOM_COHERENCE_models-4.h5',
             'comparison_mixtures-128_OC-2.0_priors-L2_L4_RANDOM_COHERENCE_models-6.h5',
             'comparison_mixtures-128_OC-2.0_priors-L2_L4_RANDOM_COHERENCE_models-COULOMB.h5',
             'comparison_mixtures-128_OC-2.0_priors-L2_L4_RANDOM_COHERENCE_models-COULOMB_F.h5',
             'comparison_mixtures-128_OC-2.0_priors-L2_L4_RANDOM_COHERENCE_models-RANDOM.h5',
             'comparison_mixtures-128_OC-2.0_priors-L2_L4_RANDOM_COHERENCE_models-RANDOM_F.h5']
models = [f.split('.')[-2].split('-')[-1] for f in fit_files]
print models

['2', '4', '6', 'COULOMB', 'COULOMB_F', 'RANDOM', 'RANDOM_F']


In [4]:
with h5py.File(os.path.join(folder, a_file), 'r') as f:
    A_array = f['A_array'].value
    A_priors = f['A_priors'].value

In [10]:
colors = ['black', 'black', 'black', 'red', 'red', 'green', 'green']
fmts = ['-', '--', ':', '-', '--', '-', '--']
with h5py.File(os.path.join(folder, fit_files[0]), 'r') as f:
    lambdas = f['lambdas'].value
n_mixtures, n_sources = A_array.shape[2:]
n_iter = A_array.shape[1]

In [6]:
W_fits = np.nan * np.ones((len(A_priors), len(models), lambdas.size, n_iter, n_sources, n_mixtures))
results = np.nan * np.ones((len(A_priors), len(models), lambdas.size, n_iter, 3))
max_angles = np.nan * np.ones((len(A_priors), len(models), lambdas.size, n_iter, n_sources))
other_angles = np.nan * np.ones((len(A_priors), len(models), lambdas.size, n_iter, n_sources**2-n_sources))

for ii, f_name in enumerate(fit_files):
    with h5py.File(os.path.join(folder, f_name), 'r') as f:
        W_fits[:, ii] = np.squeeze(f['W_fits'])

In [7]:
reload(analysis)
for ii, p in enumerate(A_priors):
    for jj, m in enumerate(models):
        for kk, l in enumerate(lambdas):
            for ll in range(n_iter):
                try:
                    A = A_array[ii, ll]
                    W = W_fits[ii, jj, kk, ll]
                    hd, (pd, ma, oa, mod) = analysis.recovery_statistics_AW(A, W)
                    results[ii, jj, kk, ll] = [hd, pd, mod]
                    max_angles[ii, jj, kk, ll] = ma
                    other_angles[ii, jj, kk, ll] = oa
                except ValueError:
                    print ii, jj, kk, ll

In [18]:
f, axes = plt.subplots(len(A_priors), 5, figsize=(3*5, 3*len(A_priors)))
for ii, (p, a) in enumerate(zip(A_priors, axes)):
    (ad, ae, ap, aa, am) = a
    for jj, m in enumerate(models):
        if ii == 0:
            label = str(m)
        else:
            label = None
        ma = np.median(max_angles[ii, jj, :, :], axis=-1)
        oa = np.median(other_angles[ii, jj, :, :], axis=-1)
        da = (ma - oa).mean(axis=1)
        dist = results[ii, jj, :, :, 0].mean(axis=1)
        delta = results[ii, jj, :, :, 1].mean(axis=1)
        distp = results[ii, jj, :, :, 2].mean(axis=1)
        ad.semilogx(lambdas, dist, fmts[jj], label=label, c=colors[jj])
        ad.semilogx(lambdas, dist, '.', c=colors[jj])
        ae.semilogx(lambdas, delta, fmts[jj], label=label, c=colors[jj])
        ae.semilogx(lambdas, delta, '.', c=colors[jj])
        ap.semilogx(lambdas, distp, fmts[jj], label=label, c=colors[jj])
        ap.semilogx(lambdas, distp, '.', c=colors[jj])
        aa.semilogx(lambdas, da, fmts[jj], label=label, c=colors[jj])
        aa.semilogx(lambdas, da, '.', c=colors[jj])
        am.semilogx(lambdas, ma.mean(axis=1), fmts[jj], label=label, c=colors[jj])
        am.semilogx(lambdas, ma.mean(axis=1), '.', c=colors[jj])
    ad.legend(loc='lower left', bbox_to_anchor=(.5, .4, 1.1, 1.1), prop={'size': 10})

    ad.set_title(p)
    ad.set_xlabel('Lambda')
    ad.set_ylabel('Hellinger Distance')
    ae.set_title(p)
    ae.set_xlabel('Lambda')
    ae.set_ylabel('Permutation Delta')
    ap.set_title(p)
    ap.set_xlabel('Lambda')
    ap.set_ylabel('Hellinger Dist. Max vs. Other')
    aa.set_title(p)
    aa.set_xlabel('Lambda')
    aa.set_ylabel('Diff. in Median Max vs. Other')
    am.set_title(p)
    am.set_xlabel('Lambda')
    am.set_ylabel('Median Max')

f.tight_layout()
plt.savefig('model_fit_comparison.pdf')
plt.show()
plt.close()

In [22]:
f, axes = plt.subplots(len(A_priors), len(models), figsize=(6*len(models), 2*len(A_priors)))
for ii, (p, a) in enumerate(zip(A_priors, axes)):
    for jj, (m, ax) in enumerate(zip(models, a)):
        if (ii == 3) and (jj == 1):
            labels = ['Max dist.', 'Max median', 'Max mode', 'Other dist.', 'Other median', 'Other mode']
        else:
            labels = 6 * [None]
        ma = max_angles[ii, jj, 4, :]
        oa = other_angles[ii, jj, 4, :]
        m_median = np.median(ma)
        o_median = np.median(oa)
        md, bins = analysis.angle_histogram(ma)
        od, bins = analysis.angle_histogram(oa)
        m_mode = md.argmax()
        o_mode = od.argmax()
        ax.semilogy(bins[:-1], md, drawstyle='steps', color='red', lw=1.5, label=labels[0])
        ax.axvline(m_median, color='red', ls='dotted', label=labels[1])
        ax.axvline(m_mode, color='red', ls='dashed', label=labels[2])
        ax.semilogy(bins[:-1], od, drawstyle='steps', color='black', lw=1.5, label=labels[3])
        ax.axvline(o_median, color='black', ls='dotted', label=labels[4])
        ax.axvline(o_mode, color='black', ls='dashed', label=labels[5])
        if (ii == 3) and (jj == 1):
            ax.legend(loc='center', prop={'size': 10})

        ax.set_title(str(m))
        if ii == len(A_priors)-1:
            ax.set_xlabel('Angle')
        if jj == 0:
            ax.set_ylabel(p)
        ax.set_xticks([0, 45, 90])
        ax.set_yticks([1e-6, 1e-3, 1e0])

f.tight_layout()
plt.savefig('permutation_comparison.pdf')
plt.show()
plt.close()

# Pickle Storage

In [3]:
fname = ''
with open(fname, 'r') as f:
    data = cPickle.load(f)

In [22]:
A_dict, W_dict, W_fits, min_ks, results = data
n_mixtures, n_sources = A_dict[A_dict.keys()[0]][0].shape
n_iter = W_fits.shape[3]
results = np.nan * np.ones((len(A_priors), len(models), lambdas.size, n_iter, 3))
max_angles = np.nan * np.ones((len(A_priors), len(models), lambdas.size, n_iter, n_sources))
other_angles = np.nan * np.ones((len(A_priors), len(models), lambdas.size, n_iter, n_sources**2-n_sources))

In [23]:
for ii, p in enumerate(A_priors):
    for jj, m in enumerate(models):
        A = A_dict[p][jj]
        for kk, l in enumerate(lambdas):
            for ll in range(n_iter):
                W = W_fits[ii, jj, kk, ll]
                hd, (pd, ma, oa, mod) = analysis.recovery_statistics_AW(A, W)
                results[ii, jj, kk, ll] = [hd, pd, mod]
                max_angles[ii, jj, kk, ll] = ma
                other_angles[ii, jj, kk, ll] = oa